
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
Root = "/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio"
os.chdir(Root)

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
#Extract features (mfcc, chroma, mel) from a sound file
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))
        if mel:
            mel=np.mean(librosa.feature.melspectrogram(y=X,sr=sample_rate).T,axis=0)
            result=np.hstack((result, mel))
    return result

In [ ]:

emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['calm', 'happy', 'fearful', 'disgust']

In [ ]:
#Load the data and extract features for each sound file
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob("/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)

In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
x_train

array([[-5.00958313e+02,  3.33561363e+01, -2.19627724e+01, ...,
         4.66570898e-04,  3.32368480e-04,  1.58667724e-04],
       [-3.68174896e+02,  1.21515379e+01, -4.14459991e+01, ...,
         5.45278145e-03,  3.56062246e-03,  2.32826290e-03],
       [-6.60813721e+02,  5.15558128e+01,  8.22291183e+00, ...,
         3.79883044e-04,  1.51239452e-04,  8.75224723e-05],
       ...,
       [-6.38889038e+02,  4.68636932e+01,  4.74644452e-03, ...,
         1.31693756e-04,  1.22964979e-04,  8.94539771e-05],
       [-5.81752563e+02,  3.85484390e+01, -1.54588423e+01, ...,
         6.08591581e-05,  4.53564789e-05,  4.73912369e-05],
       [-6.41227722e+02,  4.49487762e+01, -1.85174131e+00, ...,
         3.89261913e-05,  3.05255380e-05,  2.94166657e-05]])

In [ ]:
y_train

['happy',
 'fearful',
 'disgust',
 'calm',
 'calm',
 'disgust',
 'calm',
 'disgust',
 'calm',
 'happy',
 'calm',
 'happy',
 'happy',
 'happy',
 'calm',
 'disgust',
 'happy',
 'happy',
 'calm',
 'fearful',
 'happy',
 'happy',
 'disgust',
 'happy',
 'disgust',
 'happy',
 'calm',
 'disgust',
 'fearful',
 'calm',
 'disgust',
 'disgust',
 'happy',
 'fearful',
 'calm',
 'fearful',
 'fearful',
 'disgust',
 'disgust',
 'fearful',
 'disgust',
 'disgust',
 'fearful',
 'fearful',
 'calm',
 'disgust',
 'happy',
 'disgust',
 'happy',
 'calm',
 'disgust',
 'disgust',
 'calm',
 'happy',
 'fearful',
 'happy',
 'disgust',
 'happy',
 'disgust',
 'happy',
 'disgust',
 'disgust',
 'happy',
 'disgust',
 'calm',
 'happy',
 'happy',
 'happy',
 'disgust',
 'disgust',
 'fearful',
 'happy',
 'fearful',
 'calm',
 'disgust',
 'calm',
 'calm',
 'happy',
 'fearful',
 'calm',
 'calm',
 'fearful',
 'disgust',
 'disgust',
 'disgust',
 'happy',
 'fearful',
 'fearful',
 'disgust',
 'fearful',
 'disgust',
 'happy',
 'hap

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

(583, 195)


In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[1]}')

Features extracted: 180


In [ ]:
#Initialize the Multi Layer Perceptron Classifier
model=MLPClassifier(alpha=0.01, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=500)

In [ ]:
#Train the model
model.fit(x_train,y_train)

MLPClassifier(alpha=0.01, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=500)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

array(['disgust', 'happy', 'calm', 'fearful', 'calm', 'calm', 'happy',
       'disgust', 'fearful', 'fearful', 'fearful', 'happy', 'happy',
       'fearful', 'calm', 'calm', 'happy', 'happy', 'disgust', 'fearful',
       'disgust', 'calm', 'happy', 'happy', 'fearful', 'happy', 'calm',
       'happy', 'calm', 'happy', 'happy', 'calm', 'fearful', 'disgust',
       'calm', 'calm', 'disgust', 'calm', 'disgust', 'happy', 'happy',
       'happy', 'happy', 'fearful', 'disgust', 'fearful', 'happy', 'calm',
       'calm', 'calm', 'happy', 'happy', 'fearful', 'calm', 'calm',
       'fearful', 'happy', 'happy', 'disgust', 'happy', 'fearful',
       'fearful', 'happy', 'disgust', 'disgust', 'happy', 'fearful',
       'disgust', 'happy', 'calm', 'disgust', 'disgust', 'fearful',
       'happy', 'calm', 'disgust', 'happy', 'happy', 'happy', 'calm',
       'fearful', 'calm', 'disgust', 'happy', 'happy', 'happy', 'disgust',
       'fearful', 'calm', 'fearful', 'happy', 'calm', 'happy', 'calm',
       '

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

Accuracy: 68.72%


In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

array([0.7755102 , 0.63414634, 0.66666667, 0.66666667])

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(20)

,Actual,Predicted
0,disgust,disgust
1,fearful,happy
2,calm,calm
3,disgust,fearful
4,calm,calm
5,calm,calm
6,happy,happy
7,disgust,disgust
8,fearful,fearful
9,fearful,fearful


In [ ]:
import pickle
# Writing different model files to file
with open( 'modelForPrediction1.pkl', 'wb') as f:
    pickle.dump(model,f)

In [ ]:
filename = 'modelForPrediction1.pkl'
loaded_model = pickle.load(open(filename, 'rb')) # loading the model file from the storage

feature=extract_feature("/content/sad.wav", mfcc=True, chroma=True, mel=True)

feature=feature.reshape(1,-1)

prediction=loaded_model.predict(feature)
prediction

array(['disgust'], dtype='<U7')

In [ ]:
pip install flask-ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask import Flask, render_template
from pyngrok import ngrok

In [ ]:
port_no=5000

In [ ]:
app=Flask(__name__, template_folder="/content/drive/MyDrive/Colab Notebooks/RAVDESS_Emotional_speech_audio/templates")
ngrok.set_auth_token('2NXytsBURleu8B5mzvSuCDhTl9x_5H8YbutZGKqh3Yuoc4cZd')
public_url=ngrok.connect(port_no).public_url

@app.route('/')
def home():
  if prediction=='calm':
    return render_template("sad_Comedy.html")
  if prediction=='disgust':
    return render_template("Disguest_Musical.html")
  if prediction == 'fearful':
    return render_template("Fear_Sport.html")
  if prediction == 'happy':
    return render_template("Angry_Family entertainments.html")

print(f"to access the colab link please click",{public_url})
app.run(port=port_no)

to access the colab link please click {'http://2585-34-125-190-49.ngrok.io'}
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
